In [1]:
import tensorflow
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

from gan_models import generator, discriminator 
from gan_utils import create_dataset_from_folder

from tensorflow.keras import layers, Sequential


ModuleNotFoundError: No module named 'gan_models'

In [ ]:
def generator(latent_dim, z): # paper -> no fully connected layers, 'batchnorm in both the generator and the discriminator'
    x = Sequential()
    x.add(layers.Dense(128 * 16 * 16, input_dim = latent_dim))
    x.add(layers.LeakyReLU())
    x.add(layers.Reshape((16,16,128)))
    
    x.add(layers.Conv2D(256, 5 ,padding='same'))
    x.add(layers.LeakyReLU())
    
    x.add(layers.Conv2DTranspose(256, 4, strides=2, padding='same'))
    x.add(layers.LeakyReLU())
    
    x.add(layers.Conv2D(256, 5, padding='same'))
    x.add(layers.LeakyReLU())
    x.add(layers.Conv2DTranspose(256, 4, strides=3, padding='same'))
    x.add(layers.LeakyReLU())
    x.add(layers.Conv2D(256, 5, padding='same'))
    x.add(layers.LeakyReLU())
    
    x.add(layers.Conv2D(z, 7, activation='relu', padding='same'))
    return x


def discriminator(height, width, z): # paper -> Use LeakyReLU activation in the discriminator for all layers
    x = Sequential() #  In the LeakyReLU, the slope of the leak was set to 0.2 in all models
    x.add(layers.Conv2D(128,(3,3),input_shape=(height, width, z)))
    x.add(layers.LeakyReLU())
    
    x.add(layers.Conv2D(256,4,strides=2))
    x.add(layers.LeakyReLU())
    x.add(layers.Conv2D(256,4,strides=2))
    x.add(layers.LeakyReLU())
    x.add(layers.Conv2D(128,4,strides=2))
    x.add(layers.LeakyReLU())
    
    x.add(layers.Flatten())
    x.add(layers.Dropout(.4))
    x.add(layers.Dense(512))
    x.add(layers.Dense(1, activation='sigmoid'))
    return x

In [ ]:
def main():

    latent_dim = 32
    height = 96
    width = 96
    z = 96
    norm = 686 # No pre-processing was applied to training images besides scaling to the range of the tanh activation function [-1, 1].
    data_dir = './data/model_input/'
    save_dir = 'model/houses/'
    save_every = 10

    gen = generator(latent_dim, z)
    dis = discriminator(height, width, z)

    print('GENERATOR SUMMARY')
    print(gen.summary())
    print('DISCRIMINATOR SUMMARY')
    print(dis.summary())

    dis.compile(optimizer=Adam(lr=.00001), loss='binary_crossentropy')

    gan_input = tensorflow.keras.Input(shape=(latent_dim,))
    gan_output = dis(gen(gan_input))
    gan = tensorflow.keras.models.Model(gan_input, gan_output)
    gan.compile(optimizer=Adam(lr=.00001, clipvalue=1, decay=1e-8), loss='binary_crossentropy')



    x_train = create_dataset_from_folder(data_dir)
    print('DATASET SHAPE')
    print(x_train.shape)
    
    x_train = x_train.reshape((x_train.shape[0],) + (height, width, z)).astype('float32') / norm


    batch_size = 32
    save_dir = 'model/houses/'
    epochs = 500
    num_samples_trained = 0

    for epoch in range(1, epochs + 1):
        d_avg_loss = 0
        a_avg_loss = 0
        print(f'Epoch: {epoch}')
        for index in range(x_train.shape[0] // batch_size):
            random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
            real_images = x_train[index * batch_size: (index + 1) * batch_size]
            generated_images = gen.predict(random_latent_vectors)
            # print(generated_images[0])
            # sys.exit()
            #find a way to stitch and check 3d arrays, gonna be tough with 96z axis

            combined_images = np.concatenate([generated_images, real_images])
            labels = np.concatenate([np.ones((batch_size,1)),
                                np.zeros((batch_size, 1))])
            #labels += .05 * np.random.random(labels.shape)
            d_loss = dis.train_on_batch(combined_images, labels)
            d_avg_loss += d_loss
            random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
            misleading_targets = np.zeros((batch_size,1))
        
            dis.trainable = False
            a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
            a_avg_loss += a_loss
            dis.trainable = True

            num_samples_trained += batch_size
            
            if epoch % save_every == 0:
                e_save_dir = save_dir + f'{epoch}/'
                # add unique folder for each model run, just in case

                if os.path.exists(e_save_dir) and index == 0:
                    shutil.rmtree(e_save_dir)
                    os.makedirs(e_save_dir)
                    os.makedirs(e_save_dir + 'generated\\')
                    os.makedirs(e_save_dir + 'real\\')
                elif not os.path.exists(e_save_dir) and index == 0:
                    os.makedirs(e_save_dir)
                    os.makedirs(e_save_dir + 'generated\\')
                    os.makedirs(e_save_dir + 'real\\')
                else:
                    pass

                
                np.save(e_save_dir + 'generated\\' + 'generated_house' + str(index), generated_images[0] * norm)
                # img = image.array_to_img(generated_images[0] * 255., scale=False)
                # img.save(os.path.join(save_dir,'generated\\' + 'generated_frog' + str(step) + '.png'))
                np.save(e_save_dir + 'real\\' + 'real_house' +str(index), real_images[0] * norm)

        if epoch % save_every == 0:
            print('discriminator loss:', d_avg_loss / 82)
            print('adversarial loss:', a_avg_loss / 82)
            gan.save('model/models/gan.h5')
            gen.save('model/models/gen.h5')
            print(num_samples_trained)


In [ ]:
main()